In [ ]:
import sys
sys.path.append('..')

import torch
import os
from src.model import GPTModel
from src.loader import load_weights_into_gpt

# Lightning SDK for persistent cloud storage
try:
    from lightning_sdk import Studio, Teamspace
    
    # Use Teamspace for persistent storage (survives Studio deletion!)
    teamspace = Teamspace()
    studio = Studio()  # Keep studio for info
    
    LIGHTNING_AVAILABLE = True
    print("✓ Lightning AI SDK connected - persistent cloud storage enabled!")
    print(f"✓ Teamspace: {teamspace.name}")
    print(f"  Files uploaded to Teamspace persist even if you delete this Studio!")
except Exception as e:
    LIGHTNING_AVAILABLE = False
    print(f"Note: Lightning AI SDK not available ({e})")


def save_model_local(model, save_path, model_name="finetuned_model"):
    """
    Save the model state dict to local disk.
    
    Args:
        model: The PyTorch model to save
        save_path: Directory path where to save the model
        model_name: Name of the model file (without extension)
    
    Returns:
        str: Path to the saved model file
    """
    os.makedirs(save_path, exist_ok=True)
    model_file = os.path.join(save_path, f"{model_name}.pth")
    
    # Save model state dict
    torch.save(model.state_dict(), model_file)
    print(f"Model saved locally to: {model_file}")
    return model_file


def upload_model_to_lightning(model, model_name="finetuned_model", save_path="./models", cloud_path="models"):
    """
    Save model locally and upload to Lightning AI cloud storage.
    
    Args:
        model: The PyTorch model to save
        model_name: Name of the model file (without extension)
        save_path: Local directory for saving before upload
        cloud_path: Directory path in Lightning AI cloud storage (e.g., "models" or "checkpoints")
    
    Returns:
        str: Path to the uploaded model file
    """
    if not LIGHTNING_AVAILABLE:
        print("Warning: Lightning AI SDK not available. Model saved locally only.")
        return save_model_local(model, save_path, model_name)
    
    # Save model locally first
    model_file = save_model_local(model, save_path, model_name)
    
    # Upload to Lightning AI Studio storage
    try:
        # Studio storage paths are relative to /teamspace/studios/this_studio/
        remote_path = os.path.join(cloud_path, os.path.basename(model_file))
        teamspace.upload_file(model_file, remote_path)
        print(f"✓ Model uploaded to Lightning AI Studio cloud storage")
        print(f"  Cloud path: {remote_path}")
        print(f"  You can access this from any Lightning AI Studio session!")
    except Exception as e:
        print(f"Error uploading to Lightning AI: {e}")
        print("Model saved locally but upload failed.")
    
    return model_file


def download_model_from_lightning(model, model_name="finetuned_model", cloud_path="models", download_path="./downloaded_models"):
    """
    Download a model from Lightning AI cloud storage and load it.
    
    Args:
        model: An initialized model instance
        model_name: Name of the model file (without extension)
        cloud_path: Directory path in Lightning AI cloud storage where model is stored
        download_path: Local directory to download the model to
    
    Returns:
        model: The model with loaded weights
    """
    if not LIGHTNING_AVAILABLE:
        print("Error: Lightning AI SDK not available.")
        return model
        
    os.makedirs(download_path, exist_ok=True)
    model_file = f"{model_name}.pth"
    local_path = os.path.join(download_path, model_file)
    remote_path = os.path.join(cloud_path, model_file)
    
    try:
        # Download from Lightning AI Studio storage
        teamspace.download_file(remote_path, local_path)
        print(f"✓ Model downloaded from Lightning AI Studio cloud")
        print(f"  Downloaded from: {remote_path}")
        
        # Load the model
        model.load_state_dict(torch.load(local_path, map_location='cpu'))
        print(f"✓ Model loaded successfully")
    except Exception as e:
        print(f"Error downloading from Lightning AI: {e}")
        print(f"Make sure the model exists at: {remote_path}")
    
    return model


def save_checkpoint_with_upload(model, optimizer, epoch, loss, checkpoint_name="checkpoint", save_path="./checkpoints", cloud_path="checkpoints"):
    """
    Save a training checkpoint locally and upload to Lightning AI.
    
    Args:
        model: The PyTorch model
        optimizer: The optimizer
        epoch: Current epoch number
        loss: Current loss value
        checkpoint_name: Name of the checkpoint file (without extension)
        save_path: Directory path where to save locally
        cloud_path: Directory path in Lightning AI cloud storage
    
    Returns:
        str: Path to the checkpoint file
    """
    os.makedirs(save_path, exist_ok=True)
    checkpoint_file = os.path.join(save_path, f"{checkpoint_name}.pth")
    
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    
    # Save locally
    torch.save(checkpoint, checkpoint_file)
    print(f"Checkpoint saved locally to: {checkpoint_file}")
    
    # Upload to Lightning AI
    if LIGHTNING_AVAILABLE:
        try:
            remote_path = os.path.join(cloud_path, os.path.basename(checkpoint_file))
            teamspace.upload_file(checkpoint_file, remote_path)
            print(f"✓ Checkpoint uploaded to Lightning AI cloud: {remote_path}")
        except Exception as e:
            print(f"Note: Could not upload to Lightning AI: {e}")
    
    return checkpoint_file


def list_lightning_files(cloud_path="models"):
    """
    List files in your Lightning AI Studio cloud storage.
    
    Args:
        cloud_path: Directory path to list (e.g., "models", "checkpoints")
    """
    if not LIGHTNING_AVAILABLE:
        print("Error: Lightning AI SDK not available.")
        return
        
    print(f"Files in Lightning AI Studio cloud storage ({cloud_path}):")
    try:
        # Note: Studio uses filesystem paths, so we can just list the directory
        import glob
        pattern = os.path.join("/teamspace/studios/this_studio", cloud_path, "*")
        files = glob.glob(pattern)
        if files:
            for file in files:
                # Show relative path
                rel_path = os.path.relpath(file, "/teamspace/studios/this_studio")
                size_mb = os.path.getsize(file) / (1024 * 1024)
                print(f"  - {rel_path} ({size_mb:.2f} MB)")
        else:
            print(f"  No files found in {cloud_path}")
    except Exception as e:
        print(f"Error listing files: {e}")

# Lightning AI Model Upload & Download

This notebook helps you upload your fine-tuned models to **Lightning AI Teamspace cloud storage** and download them later from any Studio.

## 🔒 **Persistent Storage - Survives Studio Deletion!**

**Important:** We use **Teamspace storage**, not Studio storage:
- ✅ **Files persist even if you DELETE this Studio**
- ✅ Access from ANY Studio in your Teamspace
- ✅ Permanent cloud storage for your models
- ✅ No need to worry about losing your trained models!

## Available Functions:

1. **`save_model_local(model, save_path, model_name)`** - Save model locally only
2. **`upload_model_to_lightning(model, model_name, save_path, cloud_path)`** - Save and upload to Teamspace
3. **`download_model_from_lightning(model, model_name, cloud_path, download_path)`** - Download from Teamspace
4. **`save_checkpoint_with_upload(model, optimizer, epoch, loss, checkpoint_name, save_path, cloud_path)`** - Save and upload checkpoint
5. **`list_lightning_files(cloud_path)`** - List files in cloud storage

---


In [ ]:
# Upload your trained model to Lightning AI cloud storage
import os

model_file = "gpt2-large774M-sft-lora-8-16-BEST.pth"
local_path = os.path.join(".", model_file)

# Check if file exists
if not os.path.exists(local_path):
    print(f"❌ Model file not found: {local_path}")
else:
    file_size_mb = os.path.getsize(local_path) / (1024 * 1024)
    print(f"📁 Found model file: {model_file}")
    print(f"📊 File size: {file_size_mb:.2f} MB")
    
    if LIGHTNING_AVAILABLE:
        try:
            # Upload to cloud storage
            cloud_path = f"models/{model_file}"
            print(f"\n⬆️  Uploading to Lightning AI cloud...")
            # Fixed: use positional arguments instead of keyword arguments
            teamspace.upload_file(local_path, cloud_path)
            
            print(f"✅ Model successfully uploaded!")
            print(f"📍 Cloud path: {cloud_path}")
            print(f"💡 Access this model from any Lightning AI Studio session!")
        except Exception as e:
            print(f"❌ Error uploading: {e}")
    else:
        print("⚠️  Lightning AI SDK not available. Cannot upload to cloud.")


### Download This Model Later

To download and use this model in another session or Studio:


In [ ]:
# Get information about your Lightning AI Studio and uploaded models
import os
import glob

if LIGHTNING_AVAILABLE:
    print("=" * 70)
    print("🏢 LIGHTNING AI STUDIO INFORMATION")
    print("=" * 70)
    print(f"\n📍 Studio Name: {studio.name}")
    print(f"📁 Workspace Path: /teamspace/studios/this_studio/")
    print(f"\n🌐 Web Access:")
    print(f"   Go to: https://lightning.ai/studios")
    print(f"   Open your Studio: {studio.name}")
    print(f"   Use the File Browser (📁 icon) in the left sidebar")
    
    print("\n" + "=" * 70)
    print("📦 YOUR UPLOADED MODELS")
    print("=" * 70)
    
    # Check for models in the models directory
    models_dir = "/teamspace/studios/this_studio/instruct-lite/notebooks/models"
    if os.path.exists(models_dir):
        model_files = glob.glob(os.path.join(models_dir, "*.pth"))
        if model_files:
            print(f"\n✅ Found {len(model_files)} model(s) in cloud storage:\n")
            for model_file in model_files:
                file_name = os.path.basename(model_file)
                size_gb = os.path.getsize(model_file) / (1024 ** 3)
                rel_path = os.path.relpath(model_file, "/teamspace/studios/this_studio")
                print(f"   📄 {file_name}")
                print(f"      Size: {size_gb:.2f} GB")
                print(f"      Path: {rel_path}")
                print()
        else:
            print("\n⚠️  No .pth files found in models/ directory yet")
            print("   Upload a model using the cell above!")
    else:
        print(f"\n📁 Models directory will be created at: {models_dir}")
        print("   Upload your first model to create it!")
    
    print("=" * 70)
    print("\n🔒 TEAMSPACE PERSISTENT STORAGE")
    print("=" * 70)
    print(f"   Teamspace: {teamspace.name}")
    print("   ✅ Files uploaded to Teamspace storage persist FOREVER")
    print("   ✅ Access from ANY Studio in your Teamspace")
    print("   ✅ Safe even if you DELETE this Studio!")
    print("=" * 70)
else:
    print("⚠️  Lightning AI SDK not available")


In [ ]:
# Register your already-uploaded model for better web visibility
import os

if LIGHTNING_AVAILABLE:
    model_file = "gpt2-large774M-sft-lora-8-16-BEST.pth"
    local_path = os.path.join(".", model_file)
    
    if os.path.exists(local_path):
        try:
            model_name = "gpt2-large774M-sft-lora-8-16-BEST"
            model_version = "v1"
            
            print(f"📦 Registering model in Teamspace...")
            print(f"   Model name: {model_name}")
            print(f"   Version: {model_version}")
            print(f"   File size: {os.path.getsize(local_path) / (1024**3):.2f} GB")
            print(f"\n⬆️  Uploading (this creates a web-visible entry)...")
            
            # Register the model using upload_model
            result = teamspace.upload_model(
                path=local_path,
                name=model_name,
                version=model_version
            )
            
            print(f"\n✅ Model registered successfully!")
            print(f"\n🌐 VIEW ON WEB:")
            print(f"   1. Go to: https://lightning.ai/{teamspace.owner}/")
            print(f"   2. Click on: {teamspace.name} Teamspace")
            print(f"   3. Navigate to: 'Models' section")
            print(f"   4. Look for: {model_name} ({model_version})")
            print(f"\n💡 The model is now browsable in the web interface!")
            
        except Exception as e:
            print(f"❌ Error: {e}")
            print(f"\nIf the model already exists, you can:")
            print(f"  - Use a different version name")
            print(f"  - Or download using teamspace.download_file()")
    else:
        print(f"❌ Model file not found: {local_path}")
else:
    print("⚠️  Lightning AI SDK not available")


### 📋 List All Registered Models in Teamspace


In [ ]:
# List all models registered in your Teamspace (visible on web!)
if LIGHTNING_AVAILABLE:
    print("="  * 70)
    print("🎯 MODELS REGISTERED IN TEAMSPACE (Web-Visible)")
    print("=" * 70)
    
    try:
        models = teamspace.list_models()
        
        if models:
            print(f"\n✅ Found {len(models)} registered model(s):\n")
            for model in models:
                print(f"   📦 {model}")
            
            print(f"\n🌐 View these on the web:")
            print(f"   URL: https://lightning.ai/{teamspace.owner}/{teamspace.name}")
            print(f"   Section: Models")
        else:
            print("\n⚠️  No models registered yet")
            print("   Use the cell above to register your model for web visibility!")
            print(f"\n   Note: Files uploaded with upload_file() won't appear here.")
            print(f"   Use upload_model() to register models for web browsing.")
            
    except Exception as e:
        print(f"❌ Error listing models: {e}")
    
    print("\n" + "=" * 70)
else:
    print("⚠️  Lightning AI SDK not available")


---

## ⬇️ How to Download Your Model

You can download your model from any Lightning AI Studio in your Teamspace!


In [ ]:
# Download your model from Teamspace storage
from lightning_sdk import Teamspace
import os

# Connect to Teamspace
teamspace = Teamspace()

# Specify the model path and where to download it
remote_path = "models/gpt2-large774M-sft-lora-8-16-BEST.pth"
local_path = "./downloaded_models/gpt2-large774M-sft-lora-8-16-BEST.pth"

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(local_path), exist_ok=True)

print(f"⬇️  Downloading model from Teamspace...")
print(f"   Remote path: {remote_path}")
print(f"   Local path: {local_path}")
print(f"   This may take a few minutes for large files...\n")

try:
    # Download the model
    teamspace.download_file(remote_path, local_path)
    
    # Check file size
    size_gb = os.path.getsize(local_path) / (1024 ** 3)
    
    print(f"\n✅ Model downloaded successfully!")
    print(f"   📁 Location: {local_path}")
    print(f"   📊 Size: {size_gb:.2f} GB")
    print(f"\n💡 You can now load this model using PyTorch:")
    print(f"   import torch")
    print(f"   model.load_state_dict(torch.load('{local_path}'))")
    
except Exception as e:
    print(f"❌ Error downloading: {e}")
